In [1]:
import matplotlib
matplotlib.use('Qt5Agg')
from matplotlib import pyplot as plt
plt.rcParams.update({'font.size': 22})
import numpy as np

# Calculation of the second derivative between two points

second derivative at x = 0.5 is according to finite difference scheme
\begin{equation}
    f''(x_i) = \frac{ f_{i+1} - 2 f_i + f_{i-1} }{h}
\end{equation}

$f_i$ is unkown. We estimate it by linear interpolation between $x_{i+1}, x_{i-1}$.
$ m = \frac{f_{i+1}-f_{i-1}}{x_{i+1} - x_{i-1}} $ but the second derivative of a first order polynomial is zero therefore this is not helpful

Instead we can use the average of the second derivatives of the enclosing points. The motivation is, that these values should become more similiar as the points come closer together.


In [39]:
def linear_eq(x, y):
    m = (y[1]-y[0])/(x[1]-x[0])
    b = y[0] - m*x[0]
    return(m,b)

def d2_finite_diff(x,y):
    return((y[2] - 2*y[1] + y[0])/((x[2]-x[1])**2))

def predictor(x):
    x_vals = np.array([x-0.2, x, x+0.2])
    y_vals = np.exp(x_vals)
    return(x_vals, y_vals)

In [35]:
x_1 = np.array([0.3, 0.4, 0.5])
y_1 = np.exp(x_1)
d2_finite_diff(x_1, y_1)

1.493068299359069

In [36]:
x_2 = np.array([0.4, 0.5, 0.6])
y_2 = np.exp(x_2)
d2_finite_diff(x_2, y_2)

1.6500956631522854

In [38]:
(d2_finite_diff(x_1, y_1) + d2_finite_diff(x_2, y_2))/2

1.5715819812556773

In [37]:
np.exp(0.5)

1.6487212707001282

In [42]:
for i in [0.2, 0.4, 0.6]:
    x1,y1 = predictor(i)
    x2, y2 = predictor(i+0.2)
    print('Second derivative at x = {}'.format(i+0.1))
    f1 = d2_finite_diff(x1,y1)
    f2 = d2_finite_diff(x2,y2)
    mf = (f2+f1)/2
    print(mf)
    
    print(np.exp(i+0.1))
    print('Error = {}'.format(np.abs(mf-np.exp(i+0.1))))

Second derivative at x = 0.30000000000000004
1.3577446162450935
1.3498588075760032
Error = 0.007885808669090277
Second derivative at x = 0.5
1.6583530191588767
1.6487212707001282
Error = 0.00963174845874848
Second derivative at x = 0.7
2.0255169516038904
2.0137527074704766
Error = 0.011764244133413815


In [ ]:
x_exact = np.linspace(0, 1, 100)
f_exact = np.exp(x_exact)

x_calc = np.linspace(0, 1, 6)
f_calc = np.exp(x_calc)

fig, ax = plt.subplots(1,1)
ax.plot(x_exact, f_exact)
ax.scatter(x_calc, f_calc)